## rnn steps of conv_lstm.py 

In [ ]:

def rnn_step(self, t, input_x, cur_state): #
    '''
    t:time step
    input_x tensor: [b_s, in_channels, in_w, in_h]
    cur_state list: [[[b_s, c_i_channels, in_w,in_h], [b_s, h_i_channels]], [[b_s, c_i+1_channels, in_w,in_h], [b_s, h_i+1_channels]], ...]
    '''
    out_status = []
    for l range(self.n_layers):
        h_cur, c_cur = cur_state[l] #get the pre status info
        #handle the input for sequence step
        if l==0:
            inp = input_x
        else:
            inp = out_status[l-1][0] #except the begin step of lstm
        x_conv = sefl.conv_x[l](inp)
        if self.use_bn:
            x_conv = self.bn_x[t][l](x_conv)
        c_conv = self.conv_h[l](h_cur)  #*why use h_cur for other steps*
        
        i,f, o, u = torch.split((x_conv+c_conv), self.hidden_dim[l], dim=1)
        ## convLstm formula
        c = F.sigmoid(f) * c_cur + F.sigmoid(i) * F.tanh(u) #statu accumulator
        if self.use_bn:
            h = o * F.tanh(self.bn_c[t][l](c))
        else:
            h = o * F.tanh(c)
        out_status.append([h, c])
    return out_status